In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [82]:
with open('../data/country_codes_dict.pickle', 'rb') as f:
    codes = pickle.load(f)

In [83]:
df = pd.read_csv('../data/eurovision_merged_covariates_27Jan.csv')

In [84]:
# reverse the dictionary
codes = {v: k for k, v in codes.items()}

In [85]:
# variable names to lowercase and _ instead of spaces
df.columns = df.columns.str.lower().str.replace(' ', '_')
# add countries to codes in df with codes df
df['from_country_name'] = df['from_country'].map(codes)
df['country_name'] = df['country'].map(codes)

In [86]:
# how often did each country participate?
n_participations = df.groupby('country_name')['year'].nunique().sort_values(ascending=True)
# filter countries with fewer than 5 and turn into list
countries_to_remove = n_participations[n_participations < 3].index.tolist()
# remove from df
df = df[~df['country_name'].isin(countries_to_remove)]
df

,unnamed:_0,year,from_country,votes,country,own,english,other,has_border,from_country_name,country_name
0,3,1999,AT,0.0,AT,0.0,1.0,0.0,NaN,austria,austria
1,6,1999,AT,12.0,BA,0.0,0.0,1.0,NaN,austria,bosnia and herzegovina
2,7,1999,AT,0.0,BE,0.0,1.0,0.0,NaN,austria,belgium
3,11,1999,AT,0.0,CY,1.0,0.0,0.0,NaN,austria,cyprus
4,13,1999,AT,10.0,DE,0.0,0.0,1.0,1.0,austria,germany
...,...,...,...,...,...,...,...,...,...,...,...
19214,37619,2019,SM,1.0,NO,0.0,0.0,1.0,NaN,san marino,norway
19215,37623,2019,SM,0.0,RS,0.0,0.0,1.0,NaN,san marino,serbia
19216,37624,2019,SM,12.0,RU,0.0,1.0,0.0,NaN,san marino,russia
19217,37625,2019,SM,0.0,SE,0.0,1.0,0.0,NaN,san marino,sweden


Take average votes over the years for each country pair.

In [87]:
df2 = (df
    .groupby(['from_country', 'country', 'from_country_name', 'country_name'])
    #.agg({'votes': 'mean'})
    .agg(votes = ('votes', 'mean'), num_years = ('year', 'count'))
    .reset_index()
    .sort_values('votes', ascending=False)
)

In [88]:
df2

,from_country,country,from_country_name,country_name,votes,num_years
1959,TR,AZ,turkey,azerbaijan,12.000000,5
251,AZ,TR,azerbaijan,turkey,12.000000,4
1397,ME,RS,montenegro,serbia,12.000000,7
1690,RO,MD,romania,moldova,11.800000,10
488,CY,GR,cyprus,greece,11.733333,15
...,...,...,...,...,...,...
1313,MC,RU,monaco,russia,0.000000,3
1311,MC,PL,monaco,poland,0.000000,1
1911,SK,TR,slovakia,turkey,0.000000,3
1309,MC,NL,monaco,the netherlands,0.000000,1


In [89]:
# merge df2 with itself but with from_country and country swapped and from_country_name and country_name swapped
df3 = (df2
    .merge(df2.rename(columns={'from_country': 'country', 
                               'country': 'from_country', 
                               'from_country_name': 'country_name', 
                               'country_name': 'from_country_name'}), 
           on=['from_country', 'country', 'from_country_name', 'country_name', 'num_years'])
    # remove duplicates
    .drop_duplicates()
    # remove rows where from_country == country
    .query('from_country != country')
    # calculate the absolute difference in votes
    .assign(votes_diff = lambda x: abs(x['votes_x'] - x['votes_y']))
    # re-order num_years column before votes_x
    .reindex(columns=['from_country', 'country', 'from_country_name', 'country_name', 'num_years', 'votes_x', 'votes_y', 'votes_diff'])
)
df3

,from_country,country,from_country_name,country_name,num_years,votes_x,votes_y,votes_diff
0,BA,RS,bosnia and herzegovina,serbia,6,11.333333,10.166667,1.166667
1,RS,BA,serbia,bosnia and herzegovina,6,10.166667,11.333333,1.166667
2,RU,AZ,russia,azerbaijan,10,9.400000,7.900000,1.500000
3,CH,PT,switzerland,portugal,5,9.400000,2.800000,6.600000
4,MK,BG,f.y.r. macedonia,bulgaria,4,9.250000,6.250000,3.000000
...,...,...,...,...,...,...,...,...
210,AM,HU,armenia,hungary,8,0.000000,1.250000,1.250000
215,PT,PL,portugal,poland,8,0.000000,1.500000,1.500000
218,BE,MK,belgium,f.y.r. macedonia,8,0.000000,1.125000,1.125000
222,LT,AL,lithuania,albania,9,0.000000,0.111111,0.111111


In [90]:
# from_country and country contain same pairs, but in different orders
# make combined column with pairs in alphabetical order
df3['country_pair'] = df3[['from_country_name', 'country_name']].apply(lambda x: ' - '.join(sorted(x)), axis=1)
# remove duplicate country pairs
df3 = df3.drop_duplicates(subset=['country_pair'])
# remove country_pair column
df3 = df3.drop(columns=['country_pair'])

In [91]:
# high votes, low diff
top_lovers = (df3
    .query('votes_diff < 3')
    .sort_values('votes_x', ascending=False).head(5))
top_lovers

,from_country,country,from_country_name,country_name,num_years,votes_x,votes_y,votes_diff
0,BA,RS,bosnia and herzegovina,serbia,6,11.333333,10.166667,1.166667
2,RU,AZ,russia,azerbaijan,10,9.400000,7.900000,1.500000
7,SE,AU,sweden,australia,5,8.200000,7.800000,0.400000
10,BG,AT,bulgaria,austria,4,7.500000,4.750000,2.750000
15,SI,MK,slovenia,f.y.r. macedonia,7,6.285714,4.000000,2.285714


In [92]:
top_haters = (df3
    .query('votes_diff < 3')
    .sort_values('votes_x', ascending=True).head(5))
top_haters

,from_country,country,from_country_name,country_name,num_years,votes_x,votes_y,votes_diff
174,AL,LT,albania,lithuania,9,0.111111,0.000000,0.111111
169,IS,GE,iceland,georgia,7,0.285714,0.000000,0.285714
161,RS,TR,serbia,turkey,5,0.600000,0.000000,0.600000
147,BY,SI,belarus,slovenia,6,0.833333,0.000000,0.833333
138,LT,IL,lithuania,israel,13,1.000000,0.769231,0.230769


In [93]:
top_unequals = (df3
            .sort_values('votes_diff', ascending=False).head(5))
top_unequals

,from_country,country,from_country_name,country_name,num_years,votes_x,votes_y,votes_diff
6,TR,AM,turkey,armenia,5,8.800000,0.800000,8.000000
3,CH,PT,switzerland,portugal,5,9.400000,2.800000,6.600000
11,ES,IT,spain,italy,9,7.444444,1.333333,6.111111
12,FR,IT,france,italy,9,7.222222,1.666667,5.555556
17,IT,UA,italy,ukraine,7,6.142857,0.857143,5.285714
